# **Goodreads Neighborhood**


## **Imports**


In [1]:
import os
import sys
from warnings import filterwarnings
import matplotlib.pyplot as plt
from pathlib import Path
from dotenv import load_dotenv

%matplotlib inline
%load_ext autotime

load_dotenv()
filterwarnings("ignore")

module_path = os.path.abspath(os.path.join("../"))
parent_dir = os.path.dirname(os.getcwd())
if module_path not in sys.path:
    sys.path.append(module_path)

time: 1.64 ms (started: 2022-12-24 16:42:00 -05:00)


In [6]:
import pandas as pd
import ast
import networkx as nx

import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

PATH = os.getcwd()
PROJECT = str(Path(PATH).parents[0])
CLOUD_STORAGE_BUCKET = os.getenv("CLOUD_STORAGE_BUCKET")

time: 933 µs (started: 2022-12-24 16:42:17 -05:00)


## **Read**


In [3]:
!{sys.executable} -m pip install lxml

You should consider upgrading via the '/media/starscream/wheeljack/projects/ego_networks/.venv/bin/python3 -m pip install --upgrade pip' command.
time: 1.49 s (started: 2022-12-24 16:42:01 -05:00)


In [11]:
base_site = "https://www.goodreads.com/review/list/126744914?per_page=infinite&shelf=read" 
r = requests.get(base_site)
df_list = pd.read_html(r.text)
df_read = df_list[-1].copy()
df_read["title"] = df_read["title"].apply(lambda x: re.sub("^(title )", "", x).lower())
df_read["author"] = df_read["author"].apply(lambda x: re.sub("(^author )|( \*$)", "", x).lower())
df_read["date"] = pd.to_datetime(df_read["read"].apply(lambda x: re.sub("^(date read )", "", x)))
cols = ["title", "author", "date"]
df_read = df_read[cols].sort_values(by="date", ascending=False)
df_read.head(10)

,title,author,date
3,"surface detail (culture, #9)","banks, iain m.",2022-11-13
6,statistical rethinking: a bayesian course with...,"mcelreath, richard",2022-09-24
2,novacene: the coming age of hyperintelligence,"lovelock, james e.",2022-09-10
0,introducing game theory: a graphic guide,"pastine, ivan",2022-09-03
1,the spy and the traitor: the greatest espionag...,"macintyre, ben",2022-08-28
25,the irrational ape: why flawed logic puts us a...,"grimes, david robert",2022-08-14
15,atomic habits: an easy & proven way to build g...,"clear, james",2022-07-24
4,the manager's path: a guide for tech leaders n...,"fournier, camille",2022-06-25
22,emotions revealed: recognizing faces and feeli...,"ekman, paul",2022-06-19
11,the unicorn project,"kim, gene",2022-05-05


time: 1.08 s (started: 2022-12-24 18:17:19 -05:00)


In [14]:
for i in df_read.shape[0]:
    

,title,author,date
3,"surface detail (culture, #9)","banks, iain m.",2022-11-13
6,statistical rethinking: a bayesian course with...,"mcelreath, richard",2022-09-24
2,novacene: the coming age of hyperintelligence,"lovelock, james e.",2022-09-10
0,introducing game theory: a graphic guide,"pastine, ivan",2022-09-03
1,the spy and the traitor: the greatest espionag...,"macintyre, ben",2022-08-28
25,the irrational ape: why flawed logic puts us a...,"grimes, david robert",2022-08-14
15,atomic habits: an easy & proven way to build g...,"clear, james",2022-07-24
4,the manager's path: a guide for tech leaders n...,"fournier, camille",2022-06-25
22,emotions revealed: recognizing faces and feeli...,"ekman, paul",2022-06-19
11,the unicorn project,"kim, gene",2022-05-05


time: 869 ms (started: 2022-12-24 18:48:44 -05:00)


In [131]:
base_site = "https://www.goodreads.com/review/list/126744914?shelf=currently-reading" 
r = requests.get(base_site)
df_list = pd.read_html(r.text)
df_current = df_list[-1].copy()
df_current["title"] = df_current["title"].apply(lambda x: re.sub("^(title )", "", x))
df_current["author"] = df_current["author"].apply(lambda x: re.sub("(^author )|( \*$)", "", x))
df_current["date"] = pd.to_datetime(df_current["added"].apply(lambda x: re.sub("^(date added )", "", x)))
cols = ["title", "author", "date"]
df_current = df_current[cols].sort_values(by="date", ascending=False)
df_current.head(10)

,title,author,date
0,Dive Into Design Patterns,"Shvets, Alexander",2022-09-28
1,Deep Reinforcement Learning with Python: Maste...,"Ravichandiran, Sudharsan",2022-09-24
2,Information Retrieval: Implementing and Evalua...,"Büttcher, Stefan",2022-07-27
3,"Guns, Germs, and Steel: The Fates of Human Soc...","Diamond, Jared",2022-07-10


time: 804 ms (started: 2022-12-24 00:31:20 -05:00)


In [195]:
import requests
import json

# params = {'inauthor': 'jared', 'intitle': 'guns, germs, and steel'}
params = {'inauthor': 'james clear', 'intitle': 'atomic habits'.lower()}
new_params = 'q='
new_params += '+'.join('{}:{}'.format(key, value) for key, value in params.items())
r = requests.get('https://www.googleapis.com/books/v1/volumes?', params=new_params)

_ = pd.DataFrame(json.loads(r.text).get('items'))
cols = ["kind","volumeInfo"]
_ = _[cols]
_["title"] = _["volumeInfo"].apply(lambda x: x.get("title","none").lower().strip())
_["rating"] = _["volumeInfo"].apply(lambda x: int(x.get("ratingsCount", 0)))
_["subtitle"] = _["volumeInfo"].apply(lambda x: x.get("subtitle","none").lower().strip())
_["description"] = _["volumeInfo"].apply(lambda x: x.get("description","none").lower().strip())
_ = _.sort_values(by="rating", ascending=False)
# _ = _.drop_duplicates(subset=["kind"], keep="first")
cols = ["title", "subtitle", "rating", "description"]
_ = _[cols]
_.head()

,title,subtitle,rating,description
0,atomic habits,the life-changing million-copy #1 bestseller,3,the phenomenal international bestseller: 1 mil...
1,atomic habits,an easy & proven way to build good habits & br...,0,the #1 new york times bestseller. over 4 milli...
2,atomic habits (tamil),none,0,"நீங்கள் உங்கள் வாழ்க்கையை மாற்ற விரும்பினால், ..."
3,atomic habits,an easy and proven way to build good habits an...,0,none
4,james clear memoir,timeline & biography of james clear atomic hab...,0,want to know more about james clear? continue ...


time: 564 ms (started: 2022-12-24 01:03:43 -05:00)
